In [ ]:
! pip install langchain_experimental

In [13]:
import os
os.environ['DASHSCOPE_API_KEY']="sk-***"
import pandas as pd
from IPython.display import Markdown, display
# from langchain.agents import create_csv_agent
from langchain_experimental.agents import create_csv_agent

from langchain.chat_models import ChatOpenAI
from langchain.llms import OpenAI

In [9]:
# 文件来源: https://drive.google.com/uc?id=1uCSDpM7_oDrUkxKrlUl0_785jh6VW5nG
csv_file = './tmp/btc-daily-price.csv'
df = pd.read_csv(csv_file)
df.head()

,day,month,year,day_of_week,price
0,1,January,2022,Saturday,47450.2
1,2,January,2022,Sunday,47264.8
2,3,January,2022,Monday,46222.5
3,4,January,2022,Tuesday,46155.3
4,5,January,2022,Wednesday,43439.9


In [10]:
df.tail()

,day,month,year,day_of_week,price
453,30,March,2023,Thursday,28076.4
454,31,March,2023,Friday,28482.9
455,1,April,2023,Saturday,28488.0
456,2,April,2023,Sunday,28032.2
457,3,April,2023,Monday,27720.8


In [11]:
df.shape

(458, 5)

In [39]:

# from langchain_openai import ChatOpenAI
chat = ChatOpenAI(
    model='qwen-plus-latest',
    openai_api_key=os.getenv("DASHSCOPE_API_KEY"),
    openai_api_base='https://dashscope.aliyuncs.com/compatible-mode/v1',
    # max_tokens=1024
)


agent = create_csv_agent(chat, csv_file, verbose=True,allow_dangerous_code=True)


In [40]:
agent

AgentExecutor(verbose=True, agent=RunnableAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_log_to_str(x['intermediate_steps']))
})
| PromptTemplate(input_variables=['agent_scratchpad', 'input'], input_types={}, partial_variables={'df_head': '|    |   day | month   |   year | day_of_week   |   price |\n|---:|------:|:--------|-------:|:--------------|--------:|\n|  0 |     1 | January |   2022 | Saturday      | 47450.2 |\n|  1 |     2 | January |   2022 | Sunday        | 47264.8 |\n|  2 |     3 | January |   2022 | Monday        | 46222.5 |\n|  3 |     4 | January |   2022 | Tuesday       | 46155.3 |\n|  4 |     5 | January |   2022 | Wednesday     | 43439.9 |', 'tools': 'python_repl_ast - A Python shell. Use this to execute python commands. Input should be a valid python command. When using this tool, sometimes output is abbreviated - make sure it does not look abbreviated before using it in your answer.', 'tool_names': 'python_repl_ast'}, temp

In [44]:
agent.run("从2022年到2023年，增长率是多少?") #What is the % increase from 2022 to 2023?



> Entering new AgentExecutor chain...
Thought: 我需要计算2022年和2023年的平均价格，然后根据这两个值计算增长率。为此，我需要分别筛选出这两年的数据，并计算每年的平均价格。
Action: python_repl_ast
Action Input: df_2022 = df[df['year'] == 2022]; price_2022_avg = df_2022['price'].mean(); price_2022_avg28199.722739726032022年的平均价格是28199.72。接下来，我需要计算2023年的平均价格。
Action: python_repl_ast
Action Input: df_2023 = df[df['year'] == 2023]; price_2023_avg = df_2023['price'].mean(); price_2023_avg23035.0010752688142023年的平均价格是23035.00。现在我有了2022年和2023年的平均价格，我可以计算增长率。
Action: python_repl_ast
Action Input: growth_rate = ((price_2023_avg - price_2022_avg) / price_2022_avg) * 100; growth_rate-18.31479590110109我计算出了从2022年到2023年的增长率。
Final Answer: 从2022年到2023年的增长率为-18.31%。这意味着价格在这段时间内下降了约18.31%。

> Finished chain.


'从2022年到2023年的增长率为-18.31%。这意味着价格在这段时间内下降了约18.31%。'